In [11]:
N = 3
values = [[0] * (i+1) for i in range(N)]

# Assigning profit of each item
values[0][0] = 60 
values[1][0] = 30
values[1][1] = 100   
values[2][0] = 2  
values[2][1] = 5  
values[2][2] = 120  

# Assigning weight of each item
weights = [10, 20, 30]

W_capacity = 50

In [12]:
import dimod
import dimod.utilities

# https://docs.ocean.dwavesys.com/en/latest/docs_dimod/reference/generated/dimod.binary.BinaryQuadraticModel.to_ising.html

x = [dimod.Binary(f'x_{i}') for i in range(N)]

cost = - dimod.quicksum(values[i][j] * x[i] * x[j] for i in range(N) for j in range (i+1)) # values
cost -= 0.9603*(W_capacity - dimod.quicksum(weights[i] * x[i] for i in range(N)))
cost += 0.0371*(W_capacity - dimod.quicksum(weights[i] * x[i] for i in range(N)))**2

print(cost)

model = dimod.BinaryQuadraticModel(cost.linear, cost.quadratic, cost.offset, vartype='BINARY')
H = model.to_ising()

print(H)

BinaryQuadraticModel({'x_0': 7840.15, 'x_1': -140.154, 'x_2': -169.101}, {('x_1', 'x_0'): 712.0, ('x_2', 'x_0'): 1111.0, ('x_2', 'x_1'): 39.52}, 44.735, 'BINARY')
({'x_1': 117.803, 'x_0': 4375.825, 'x_2': 203.0795}, {('x_0', 'x_1'): 178.0, ('x_2', 'x_1'): 9.88, ('x_2', 'x_0'): 277.75}, 4275.8125)


In [13]:
from neal import SimulatedAnnealingSampler


sampleset = SimulatedAnnealingSampler().sample(model, num_reads=1000)

solution = sampleset.first.sample
print(solution)

{'x_0': 0, 'x_1': 1, 'x_2': 1}


In [5]:
"""Adiabatic evolution for the Ising Hamiltonian using linear scaling."""

import argparse
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from qibo import callbacks, hamiltonians, models

matplotlib.rcParams["mathtext.fontset"] = "cm"
matplotlib.rcParams["font.family"] = "STIXGeneral"
matplotlib.rcParams["font.size"] = 14


parser = argparse.ArgumentParser()
parser.add_argument("--nqubits", default=4, type=int)
parser.add_argument("--hfield", default=1, type=float)
parser.add_argument("--T", default=1, type=float)
parser.add_argument("--dt", default=1e-2, type=float)
parser.add_argument("--solver", default="exp", type=str)
parser.add_argument("--save", action="store_true")


def main(nqubits, hfield, T, dt, solver, save):
    """Performs adiabatic evolution with critical TFIM as the "hard" Hamiltonian.

    Plots how the <H1> energy and the overlap with the actual ground state
    changes during the evolution.
    Linear scheduling is used.

    Args:
        nqubits (int): Number of qubits in the system.
        hfield (float): Transverse field Ising model h-field h value.
        T (float): Total time of the adiabatic evolution.
        dt (float): Time step used for integration.
        solver (str): Solver used for integration.
        save (bool): Whether to save the plots.
    """
    h0 = hamiltonians.X(nqubits)
    h1 = hamiltonians.TFIM(nqubits, h=hfield)
    bac = h1.backend

    # Calculate target values (H1 ground state)
    target_state = h1.ground_state()
    target_energy = bac.to_numpy(h1.eigenvalues()[0]).real

    # Check ground state
    state_energy = bac.to_numpy(h1.expectation(target_state)).real
    np.testing.assert_allclose(state_energy.real, target_energy)

    energy = callbacks.Energy(h1)
    overlap = callbacks.Overlap(target_state)

    # SIMULATED ANNEALING
    evolution = models.AdiabaticEvolution(
        h0, h1, lambda t: t, dt=dt, solver=solver, callbacks=[energy, overlap]
    )
    final_psi = evolution(final_time=T)

    # Plots
    tt = np.linspace(0, T, int(T / dt) + 1)
    plt.figure(figsize=(12, 4))
    plt.subplot(121)
    plt.plot(tt, energy[:], linewidth=2.0, label="Evolved state")
    plt.axhline(y=target_energy, color="red", linewidth=2.0, label="Ground state")
    plt.xlabel("$t$")
    plt.ylabel("$H_1$")
    plt.legend()

    plt.subplot(122)
    plt.plot(tt, overlap[:], linewidth=2.0)
    plt.xlabel("$t$")
    plt.ylabel("Overlap")

    if save:
        out_fol = Path("images")
        out_fol.mkdir(exist_ok=True)
        plt.savefig(out_fol / f"dynamics_n{nqubits}T{T}.png", bbox_inches="tight")
    else:
        plt.show()


if __name__ == "__main__":
    args = vars(parser.parse_args())
    main(**args)

usage: ipykernel_launcher.py [-h] [--nqubits NQUBITS] [--hfield HFIELD]
                             [--T T] [--dt DT] [--solver SOLVER] [--save]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\mique\AppData\Roaming\jupyter\runtime\kernel-v2-24072djB3llYh0KqZ.json


SystemExit: 2

C:\Users\mique\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
